In [ ]:
! pip install transformers sentencepiece datasets

In [5]:
!wget https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json

--2023-05-06 09:38:28--  https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.231.96, 52.216.10.117, 52.216.94.157, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.231.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209850483 (200M) [application/json]
Saving to: ‘personachat_self_original.json’

personachat_self_or 100%[===================>] 200.13M  5.40MB/s    in 22s     

2023-05-06 09:38:51 (9.01 MB/s) - ‘personachat_self_original.json’ saved [209850483/209850483]



In [8]:
import json

# Load Persona-Chat dataset from JSON file
with open('personachat_self_original.json', 'r') as f:
    dataset = json.load(f)

# Convert dataset to text file
with open('persona-chat.txt', 'w') as f:
    for conversation in dataset['train']:
        for message in conversation['utterances']:
            for m in message['candidates']:
              text = m.strip().replace('\n', ' ')
              if text:
                f.write(text + '\n')

In [1]:
with open('persona-chat.txt', 'r') as f:
  lines = f.readlines()
with open('persona-chat-10k.txt', 'w') as f:
  for i in range(10000):
    f.write(lines[i])

In [10]:
with open('persona-chat-10k.txt', 'r') as f:
  lines = f.readlines()

print("size: ", len(lines))
for i in lines[:10]:
  print(i)

size:  10000
my mom was single with 3 boys , so we never left the projects .

i try to wear all black every day . it makes me feel comfortable .

well nursing stresses you out so i wish luck with sister

yeah just want to pick up nba nfl getting old

i really like celine dion . what about you ?

no . i live near farms .

i wish i had a daughter , i'm a boy mom . they're beautiful boys though still lucky

yeah when i get bored i play gone with the wind my favorite movie .

hi how are you ? i'm eating dinner with my hubby and 2 kids .

were you married to your high school sweetheart ? i was .



In [8]:
import transformers
from tokenizers import ByteLevelBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import datasets
from datasets import load_dataset

In [12]:
paths = ['persona-chat-10k.txt']

In [13]:
import os.path
from os import path

if path.exists('/content/myTokenizer') == False:
  os.mkdir('/content/myTokenizer')
  tokenizer = ByteLevelBPETokenizer()
  tokenizer.train(files = paths, min_frequency = 2)
  tokenizer.save_model("myTokenizer")

In [14]:
tokenizer = GPT2Tokenizer.from_pretrained("myTokenizer")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [15]:
inp = "Hello world!"
t = tokenizer.encode(inp)

In [16]:
print(t)
print(tokenizer.decode(t))

[39, 461, 78, 1064, 0]
Hello world!


In [17]:
config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    bos_token=tokenizer.bos_token_id,
    eos_token=tokenizer.eos_token_id
)

model = GPT2LMHeadModel(config)
dataset = load_dataset("text", data_files=paths)
def encode(lines):
    return tokenizer(lines['text'], add_special_tokens = True, padding=True, truncation = True, max_length = 512)
dataset.set_transform(encode)
dataset = dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-9b79723d124556f1/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, mlm_probability=0.15)

In [19]:
training_args = TrainingArguments(
    output_dir='results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,6.329900
1000,5.819300
1500,5.575900
2000,5.409100
2500,5.230000
3000,5.294600
3500,5.196800
4000,5.122400
4500,5.170800
5000,5.128500


TrainOutput(global_step=30000, training_loss=4.63833623046875, metrics={'train_runtime': 26243.5304, 'train_samples_per_second': 1.143, 'train_steps_per_second': 1.143, 'total_flos': 185328518400000.0, 'train_loss': 4.63833623046875, 'epoch': 3.0})

In [20]:
if path.exists('/content/myChatBotWithGPT2Model') == False:
  os.mkdir('/content/myChatBotWithGPT2Model')
  trainer.save_model("myChatBotWithGPT2Model")

In [21]:
!zip -r myChatBotWithGPT2Model.zip myChatBotWithGPT2Model
!zip -r myTokenizer.zip myTokenizer
from google.colab import files

files.download('myChatBotWithGPT2Model.zip')
files.download('myTokenizer.zip')


  adding: myChatBotWithGPT2Model/ (stored 0%)
  adding: myChatBotWithGPT2Model/config.json (deflated 52%)
  adding: myChatBotWithGPT2Model/pytorch_model.bin (deflated 10%)
  adding: myChatBotWithGPT2Model/generation_config.json (deflated 24%)
  adding: myChatBotWithGPT2Model/training_args.bin (deflated 49%)
  adding: myTokenizer/ (stored 0%)
  adding: myTokenizer/vocab.json (deflated 59%)
  adding: myTokenizer/merges.txt (deflated 57%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [35]:
model = GPT2LMHeadModel.from_pretrained('myChatBotWithGPT2Model')
tokenizer = GPT2Tokenizer.from_pretrained("myTokenizer")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# trainer_model = trainer.model
model.config.pad_token_id = model.config.eos_token_id
while True:
  prompt = input('You: ')
  if not prompt:
    break
  print("You: ", prompt)
  model.eval()

  input_ids = tokenizer.encode(prompt,  return_tensors='pt')
  # attention_mask = torch.ones_like(input_ids)
  beam_output = model.generate(
    input_ids,
    # attention_mask=attention_mask, 
    max_length=50, 
    num_beams=5, 
    temperature=0.7, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 3,
    return_dict_in_generate = True,
    output_scores = True,
  )
  loss = beam_output['sequences_scores']
  for idx, beam in enumerate(beam_output['sequences']):
    response = tokenizer.decode(beam, skip_special_tokens=True)
    print('Bot: ' + response.replace(prompt, ""))
    print("loss:", loss[idx])


You:  hello it is rain today


c:\Users\ngohu\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Bot: . what do you do for fun? i like to do?.. how are you like??!. i'm living? what is i am you live? how about you work? my favorite? any? do
loss: tensor(-1.3245)
Bot: . what do you do for fun? i like to do?.. how are you like??!. i'm living? what is i am you live? how about you enjoy i love? do not like living
loss: tensor(-1.3355)
Bot: . what do you do for fun? i like to do?.. how are you like??!. i'm living? what is i am you live? how about you work? my favorite? any? you
loss: tensor(-1.3357)
You:  i lived in HCM city
Bot: . what do you do for fun? i'm a living?? you like?.. i love. do not work? what about you have you live? do? my favorite! i like to do
loss: tensor(-1.2596)
Bot: . what do you do for fun? i'm a living?? you like?.. i love. do not work? what about you have you live? do? my favorite! i have not like
loss: tensor(-1.2631)
Bot: . what do you do for fun? i'm a living?? you like?.. i love. do not work? what about you have you live? do? my favorite! i do. you
loss: te

## T5

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config, DataCollatorForLanguageModeling, Trainer, TrainingArguments, AdamW

import sentencepiece
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer
import datasets
from datasets import load_dataset
import os.path
from os import path
paths = ['persona-chat-10k.txt']
# train sentencepiece from persona-chat-10k.txt and make tokenizerT5/tokenizerT5.model, tokenizerT5.vocab
SentencePieceTrainer.Train('--input=persona-chat-10k.txt --model_prefix=tokenizerT5/tokenizerT5 --vocab_size=400')
# make segmenter instance and load model
sp = SentencePieceProcessor()
sp.load('tokenizerT5/tokenizerT5.model')

inp = "Hello world!"
enc = sp.encode_as_ids(inp)
enc_pie = sp.encode_as_pieces(inp)
print(enc)
print(enc_pie)
print(sp.decode_ids(enc))
print(sp.decode_pieces(enc_pie))
tokenizerT5 = T5Tokenizer.from_pretrained('tokenizerT5/tokenizerT5.model')
# tokenizerT5.pad_token = tokenizerT5.eos_token
# tokenizerT5.add_special_tokens({'pad_token': '[PAD]'})

inp = "Hello world!"
t = tokenizerT5.encode(inp, return_tensors='pt')

print(t)
print(tokenizerT5.decode(t[0]))


configT5 = T5Config(
    vocab_size=tokenizerT5.vocab_size+100,
    bos_token=tokenizerT5.bos_token_id,
    eos_token=tokenizerT5.eos_token_id,
    decoder_start_token_id=tokenizerT5.pad_token_id,
)
modelT5 = T5ForConditionalGeneration(configT5)
datasetT5 = load_dataset("text", data_files=paths)
def encode(lines):
    return tokenizerT5(lines['text'], add_special_tokens = True, padding="max_length", truncation = True, max_length = 128)
datasetT5.set_transform(encode)
datasetT5 = datasetT5['train']

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizerT5, mlm=False, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir='resultsT5',
    num_train_epochs=2,
    per_device_train_batch_size=50,
    prediction_loss_only=True,
    remove_unused_columns=False,
)

trainerT5 = Trainer(
    model=modelT5,
    args=training_args,
    train_dataset=datasetT5,
    data_collator=data_collator,
)

trainerT5.train()

In [3]:
if path.exists('myChatBotWithT5') == False:
  os.mkdir('myChatBotWithT5')
  trainerT5.save_model("myChatBotWithT5")

In [6]:
modelT5 = T5ForConditionalGeneration.from_pretrained('myChatBotWithT5')
tokenizerT5 = T5Tokenizer.from_pretrained('tokenizerT5/tokenizerT5.model')
while True:
  prompt = input('You: ')
  if not prompt:
    break
  print("You: ", prompt)
  modelT5.eval()

  input_ids = tokenizerT5.encode(prompt,  return_tensors='pt')
  beam_output = modelT5.generate(
    input_ids,
    max_length=50, 
    num_beams=5, 
    temperature=0.7, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 3,
    return_dict_in_generate = True,
    output_scores = True,
  )
  loss = beam_output['sequences_scores']
  for idx, beam in enumerate(beam_output['sequences']):
    response = tokenizerT5.decode(beam, skip_special_tokens=True)
    print('Bot: ' + response.replace(prompt, ""))
    print("loss:", loss[idx])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


You:  hello
Bot: ok okverver..<extra_id_-11><extra_id_-11>.stst 3 3<extra_id_-38><extra_id_-38>keke   i i ok. ok 3<extra_id_-84><extra_id_-84>ver<extra_id_-11> i
loss: tensor(-3.3387)
Bot: ok okverver..<extra_id_-11><extra_id_-11>.stst 3 3<extra_id_-38><extra_id_-38>keke   i i ok. ok 3<extra_id_-84><extra_id_-84><extra_id_-11>ver i
loss: tensor(-3.3389)
Bot: ok okverver..<extra_id_-11><extra_id_-11>.stst 3 3<extra_id_-38><extra_id_-38>keke   i i ok. ok 3<extra_id_-84><extra_id_-84>ver<extra_id_-11> i
loss: tensor(-3.3587)
You:  it is rain today
Bot: ..verver year yearss   out out.sversese. year sse out year.s. out i i<extra_id_-84><extra_id_-84>
loss: tensor(-3.6793)
Bot: ..verver year yearss   out out.sversese. year sse out year.s. out i i it it
loss: tensor(-3.6838)
Bot: ..verver year yearss   out out.sversese. year sse out year.s. out i i what what
loss: tensor(-3.6851)


## Reformer

In [73]:
from transformers import ReformerModelWithLMHead, ReformerTokenizer, ReformerConfig, DataCollatorForLanguageModeling, TrainingArguments, Trainer
import os.path
from os import path
from datasets import load_dataset
import sentencepiece
from sentencepiece import SentencePieceProcessor, SentencePieceTrainer

paths = ['persona-chat-10k.txt']
SentencePieceTrainer.Train('--input=persona-chat-10k.txt --model_prefix=tokenizerReformer/tokenizerReformer --vocab_size=400')
# make segmenter instance and load model
sp = SentencePieceProcessor()
sp.load('tokenizerReformer/tokenizerReformer.model')

tokenizerReformer = ReformerTokenizer.from_pretrained("tokenizerReformer/tokenizerReformer.model")
tokenizerReformer.pad_token = tokenizerReformer.eos_token
tokenizerReformer.add_special_tokens({'pad_token': '[PAD]'})


inp = "Hello world!"
t = tokenizerReformer.encode(inp, return_tensors='pt')

print(t)
print(tokenizerReformer.decode(t[0]))


reformerConfig = ReformerConfig(
    vocab_size=tokenizerReformer.vocab_size+100,
    bos_token=tokenizerReformer.bos_token_id,
    eos_token=tokenizerReformer.eos_token_id,
    decoder_start_token_id=tokenizerReformer.pad_token_id,
    is_decoder=True,
    axial_pos_shape=[16,64],
    num_buckets = 64,
)
modelReformer = ReformerModelWithLMHead(reformerConfig)
datasetReformer = load_dataset("text", data_files=paths)
def encode(lines):
    return tokenizerReformer(lines['text'], add_special_tokens = False, padding='max_length', truncation = True, max_length = 64*16)
datasetReformer.set_transform(encode)
datasetReformer = datasetReformer['train']

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizerReformer, mlm=False, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir='resultsReformer',
    num_train_epochs=3,
    per_device_train_batch_size=50,
    prediction_loss_only=True,
    remove_unused_columns=False,
)

trainerReformer = Trainer(
    model=modelReformer,
    args=training_args,
    train_dataset=datasetReformer,
    data_collator=data_collator,
)

trainerReformer.train()

tensor([[  3,   0, 301,  42,  92,  44,  86, 397]])
<unk>ello world!


Found cached dataset text (C:/Users/ngohu/.cache/huggingface/datasets/text/default-e1039025eb1bea90/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

{'train_runtime': 427.2972, 'train_samples_per_second': 0.702, 'train_steps_per_second': 0.014, 'train_loss': 6.249341328938802, 'epoch': 3.0}


TrainOutput(global_step=6, training_loss=6.249341328938802, metrics={'train_runtime': 427.2972, 'train_samples_per_second': 0.702, 'train_steps_per_second': 0.014, 'train_loss': 6.249341328938802, 'epoch': 3.0})

In [39]:
if path.exists('myChatBotWithReformer') == False:
  os.mkdir('myChatBotWithReformer')
trainerReformer.save_model("myChatBotWithReformer")

In [81]:
modelReformer = ReformerModelWithLMHead.from_pretrained('myChatBotWithReformer')
tokenizerReformer = ReformerTokenizer.from_pretrained("tokenizerReformer/tokenizerReformer.model")

while True:
  prompt = input('You: ')
  if not prompt:
    break
  print("You: ", prompt)
  modelReformer.eval()

  input_ids = tokenizerReformer.encode(prompt,  return_tensors='pt')
  beam_output = modelReformer.generate(
    input_ids,
    max_length=50, 
    num_beams=5, 
    temperature=0.7, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 3,
    return_dict_in_generate = True,
    output_scores = True,
  )
  loss = beam_output['sequences_scores']
  for idx, beam in enumerate(beam_output['sequences']):
    # loại bỏ special token
    ids = []
    for i in beam:
      if i < tokenizerReformer.vocab_size:
        ids.append(i)
      else:
        ids.append(0)
    response = tokenizerReformer.decode(ids, skip_special_tokens=True)
    print('Bot: ', response.replace(prompt, ""))
    print("loss:", loss[idx])

You:  hello
response:  helloonereoodganirere talkcount go ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
Bot:  onereoodganirere talkcount go ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
loss: tensor(-4.7078)
response:  helloonereoodganirere talkcount al ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
Bot:  onereoodganirere talkcount al ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
loss: tensor(-4.7078)
response:  hellooneganioodganirere talkcount go ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
Bot:  oneganioodganirere talkcount go ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi movrets. controlre
loss: tensor(-4.7080)
You:  it is rain today
response:  it is rain todayre talkcount go ofre takit?countre m st troneyctavvereugh winldive a to dogrere licfi mo